In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=bb30ef5f9415aea4066481b1fcbc661814d5f565a5dd982e24665b5e4a34ec23
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_4")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 4
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [4]:
from pyspark.sql import Window
from pyspark.sql import functions as F 
from pyspark.sql.pandas.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType

import pandas as pd
import requests

In [5]:
train_response = requests.get('https://drive.google.com/uc?id=1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp')
with open('raw_sales.csv', 'wb') as file:
    file.write(train_response.content)
data = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)
data.show(5)
data.printSchema()

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
|2007-02-27 00:00:00|    2906|290000|       house|       3|
|2007-03-07 00:00:00|    2905|328000|       house|       3|
|2007-03-09 00:00:00|    2905|380000|       house|       4|
|2007-03-21 00:00:00|    2906|310000|       house|       3|
+-------------------+--------+------+------------+--------+
only showing top 5 rows

root
 |-- datesold: timestamp (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- bedrooms: integer (nullable = true)



## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
*  Стоимость последнего проданного дома до текущего


In [6]:
window = Window.partitionBy('postcode').orderBy('datesold')

df = (
    data.withColumn('avg_price_before_sale', F.avg(F.col('price'))
    .over(window.rowsBetween(Window.currentRow-10, Window.currentRow-1)))
)

df = (
    df.withColumn('avg_price_after_sale', F.avg(F.col('price'))
    .over(window.rowsBetween(Window.currentRow+1, Window.currentRow+10)))
)

df = df.withColumn('last_price_before_sale', F.lag(F.col('price'), 1).over(window))

df = df.filter(F.col('propertyType') == 'house')

df = df.selectExpr('datesold', 'postcode', 'price', 'propertyType', 'bedrooms',
                   'cast(avg_price_before_sale as int) as avg_price_before_sale', 
                   'cast(avg_price_after_sale as int) as avg_price_after_sale',
                   'last_price_before_sale')

(
    df.filter(F.col('avg_price_before_sale').isNotNull())
    .select('*').orderBy('postcode', 'datesold').show(10)
)

+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|
+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+
|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|               479750|              868450|                329000|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|               520500|              805750|        

## Задание 2
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [7]:
@F.udf(returnType=IntegerType())
def count_unique_udf(row):
    return len(set(row))
(
    df.filter(F.col('avg_price_before_sale').isNotNull())
    .withColumn('unique_values', count_unique_udf(F.array(*df))).show(10)
)

+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|unique_values|
+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+
|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|            7|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|            8|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|            8|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|               479750|              868450|                329000|            8|
|2008-07-29 0

In [8]:
@pandas_udf(IntegerType(), PandasUDFType.SCALAR)
def count_unique_pandas_udf(series) -> int:
    return series.apply(lambda row: len(set(row)))
(
    df.filter(F.col('avg_price_before_sale').isNotNull())
    .withColumn('unique_values', count_unique_pandas_udf(F.array(*df))).show(10)
)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|unique_values|
+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+
|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|            7|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|            8|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|            8|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|               479750|              868450|                329000|            8|
|2008-07-29 0

# Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [9]:
df = df.withColumn(
    'sign',
    F.when(
        F.col('avg_price_before_sale').isNull() | 
        F.col('avg_price_after_sale').isNull(),
        'Нет данных'
    ).when(
        F.col('avg_price_before_sale') > F.col('avg_price_after_sale'),
        '-'
    ).when(
        F.col('avg_price_before_sale') < F.col('avg_price_after_sale'),
        '+'
    ).otherwise(
        '='
    )
)

df.show()     

+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|      sign|
+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                 null|              708350|                  null|Нет данных|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|         +|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|         +|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|         +|
|2008-06-19 00:00:00|    2600| 765